Read in and download MEPS data

In [1]:
import sys
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
#import fill_values as fv
#import calc_station_properties as cs

#import createFolder as cF
from scipy.integrate import simps

In [2]:
thredds      = 'http://thredds.met.no/thredds/dodsC/meps25epsarchive'
year         = 2016
month        = 12
day          = 24
forecasttime = '00'

In [3]:
fn = netCDF4.Dataset('%s/%s/%s/%s/meps_subset_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,forecasttime),
                     'r')
#fn = netCDF4.Dataset('%s/%s/%s/%s/meps_mbr0_full_backup_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,forecasttime),
 #                    'r')
#fn = netCDF4.Dataset('%s/%s/%s/%s/meps_mbr0_extracted_backup_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,forecasttime),
 #                    'r')

#fn = netCDF4.Dataset('%s/%s/%s/%s/meps_mbr0_pp_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,forecasttime),
 #                    'r')
#fn = netCDF4.Dataset('%s/%s/%s/%s/meps_mbr0_vc_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,forecasttime),
 #                    'r')

In [4]:
fn

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    Conventions: CF-1.6
    institution: Norwegian Meteorological Institute, MET Norway
    creator_url: met.no
    summary: MEPS (MetCoOp-Ensemble Prediction System) is a convection-permitting atmosphere ensemble model covering Scandinavia and the Nordic Seas. It has horizontal resolution of 2.5 km, has 65 vertical levels and 10 members. MEPS is ran four times daily (00,06,12,18) with three-hourly cycling for data assimilation. Member 0 and 1 are ran up to 66 hours, the rest up to 48 hours. Boundary data is from ECMWF, and initial perturbations are based on the SLAF method. Model code based on HARMONIE cy40h1.1
    source: MEPS 2.5km
    min_time: 2016-12-24 00:00:00Z
    max_time: 2016-12-26
    geospatial_lat_min: 51.0
    geospatial_lat_max: 88.0
    geospatial_lon_min: -20.0
    geospatial_lon_max: 80.0
    references: unknown
    license: http://met.no/sb5-met.no/Forskning/Informasjonste

In [5]:
def find_station_yx(latitude, longitude, stn_lat, stn_lon):
# find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
# This tells us how close a point is to the particular latitude and longitude.
    abslat = np.abs(latitude[:,:]-stn_lat)
    abslon= np.abs(longitude[:,:]-stn_lon)

# Now we need to combine these two results. We will use numpy.maximum, which takes two arrays and finds the local 
# maximum.
    c = np.maximum(abslon, abslat)

# If you don't like flattened arrays, you can also get the row/column index like this
    y, x = np.where(c == np.min(c))
    return(x,y);

In [6]:
## Latitudes
## [y = 949][x = 739]
latitude = fn.variables['latitude']

## Longitudes 
## [y = 949][x = 739]
longitude = fn.variables['longitude']

In [7]:
stn_lat = 59.8
stn_lon = 7.2

In [8]:
# Now find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
x,y = find_station_yx(latitude, longitude, stn_lat, stn_lon)


In [9]:
fn.variables['surface_air_pressure']

<class 'netCDF4._netCDF4.Variable'>
float32 surface_air_pressure(time, height0, ensemble_member, y, x)
    grid_mapping: projection_lambert
    coordinates: longitude latitude
    standard_name: surface_air_pressure
    units: Pa
    _FillValue: 9.96921e+36
    long_name: Surface air pressure
    _ChunkSizes: [  1   1   1 949 739]
unlimited dimensions: time
current shape = (67, 1, 10, 949, 739)
filling off

In [12]:
fn.variables['surface_air_pressure'][:,:,3,y[0],x[0]]

masked_array(
  data=[[87145.25],
        [87233.0],
        [87282.75],
        [87321.8125],
        [87364.1875],
        [87409.6875],
        [87410.875],
        [87402.0],
        [87459.125],
        [87455.8125],
        [87391.75],
        [87338.625],
        [87285.125],
        [87315.3125],
        [87248.625],
        [87224.25],
        [87215.8125],
        [87199.0],
        [87247.75],
        [87211.0625],
        [87193.6875],
        [87197.125],
        [87438.8125],
        [87341.3125],
        [87290.125],
        [87264.4375],
        [87390.25],
        [87408.75],
        [87416.8125],
        [87378.0625],
        [87436.75],
        [87483.25],
        [87481.375],
        [87506.6875],
        [87542.8125],
        [87476.625],
        [87421.1875],
        [87444.1875],
        [87541.75],
        [87532.1875],
        [87543.0],
        [87424.5625],
        [87417.1875],
        [87428.5625],
        [87335.75],
        [87332.4375],
        [87408.43

In [13]:
np.ma.is_masked(fn.variables['surface_air_pressure'][:,:,3,y[0],x[0]])

True

In [15]:
mask = np.ma.getmaskarray(fn.variables['surface_air_pressure'][:,:,3,y[0],x[0]])   
mask

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [18]:
marr = np.ma.array(fn.variables['surface_air_pressure'][:,:,3,y[0],x[0]], mask=mask, fill_value=np.nan)
marr

masked_array(
  data=[[87145.25],
        [87233.0],
        [87282.75],
        [87321.8125],
        [87364.1875],
        [87409.6875],
        [87410.875],
        [87402.0],
        [87459.125],
        [87455.8125],
        [87391.75],
        [87338.625],
        [87285.125],
        [87315.3125],
        [87248.625],
        [87224.25],
        [87215.8125],
        [87199.0],
        [87247.75],
        [87211.0625],
        [87193.6875],
        [87197.125],
        [87438.8125],
        [87341.3125],
        [87290.125],
        [87264.4375],
        [87390.25],
        [87408.75],
        [87416.8125],
        [87378.0625],
        [87436.75],
        [87483.25],
        [87481.375],
        [87506.6875],
        [87542.8125],
        [87476.625],
        [87421.1875],
        [87444.1875],
        [87541.75],
        [87532.1875],
        [87543.0],
        [87424.5625],
        [87417.1875],
        [87428.5625],
        [87335.75],
        [87332.4375],
        [87408.43

In [20]:
marr.filled()

array([[87145.25 ],
       [87233.   ],
       [87282.75 ],
       [87321.81 ],
       [87364.19 ],
       [87409.69 ],
       [87410.875],
       [87402.   ],
       [87459.125],
       [87455.81 ],
       [87391.75 ],
       [87338.625],
       [87285.125],
       [87315.31 ],
       [87248.625],
       [87224.25 ],
       [87215.81 ],
       [87199.   ],
       [87247.75 ],
       [87211.06 ],
       [87193.69 ],
       [87197.125],
       [87438.81 ],
       [87341.31 ],
       [87290.125],
       [87264.44 ],
       [87390.25 ],
       [87408.75 ],
       [87416.81 ],
       [87378.06 ],
       [87436.75 ],
       [87483.25 ],
       [87481.375],
       [87506.69 ],
       [87542.81 ],
       [87476.625],
       [87421.19 ],
       [87444.19 ],
       [87541.75 ],
       [87532.19 ],
       [87543.   ],
       [87424.56 ],
       [87417.19 ],
       [87428.56 ],
       [87335.75 ],
       [87332.44 ],
       [87408.44 ],
       [87378.06 ],
       [87328.31 ],
       [      nan],


In [9]:
### variables to calculate pressure
#p0 = fn.variables['p0']    ## p0: p0
ap = fn.variables['ap']    ## ap: ap
b = fn.variables['b']      ## b: b

surface_air_pressure = fn.variables['surface_air_pressure']

In [10]:
p_interface = ap[:]+b[:]*surface_air_pressure[:,:,0,y[0],x[0]]


In [11]:
p_interface.shape

(67, 5)

In [12]:
surface_air_pressure[:,:,0,y[0],x[0]].shape

(67, 1)

In [28]:
### 1) Connect interface values and surface values for pressure
### Pressure
p_interfaces2 = np.concatenate((p_interface[:,:],surface_air_pressure[:,:,0,y[0],x[0]]),axis=1)

In [29]:
pd.DataFrame(p_interfaces2)

,0,1,2,3,4,5
0,86012.144428,86294.528927,86564.799934,86827.083570,87085.889464,87215.0000
1,86099.361870,86382.033054,86652.578120,86915.127716,87174.196044,87303.4375
2,86087.835608,86370.468904,86640.977752,86903.492200,87162.525846,87291.7500
3,86110.949771,86393.659044,86664.240522,86926.825455,87185.928650,87315.1875
4,86167.903069,86450.799548,86721.559988,86984.318593,87243.593158,87372.9375
5,86195.270238,86478.256673,86749.103108,87011.945166,87271.302078,87400.6875
6,86235.951165,86519.071318,86790.045583,87053.011694,87312.491013,87441.9375
7,86156.130255,86438.988036,86709.711484,86972.434189,87231.673330,87361.0000
8,86223.130510,86506.208521,86777.142500,87040.069515,87299.510257,87428.9375
9,86295.678163,86578.994639,86850.156581,87113.304823,87372.963858,87502.5000


In [37]:
# transform hybrid sigma pressure coordinates at interface levels to pressure at model levels
p_ml = np.empty([p_interfaces2.shape[0],p_interfaces2.shape[1]-1])
for i in range(0,p_interfaces2.shape[1]-1):
    p_ml[:,i] = 1/2 * (p_interfaces2[:,i] + p_interfaces2[:,i+1])
p_ml = np.concatenate((p_ml[:,:],surface_air_pressure[:,:,0,y[0],x[0]]),axis=1)
### --> Now the pressure is calculated at each model level

In [38]:
p_ml[:,:].shape

(67, 6)

In [39]:
pd.DataFrame(p_ml[:,:])

,0,1,2,3,4,5
0,86153.336678,86429.664431,86695.941752,86956.486517,87150.444732,87215.0000
1,86240.697462,86517.305587,86783.852918,87044.661880,87238.816772,87303.4375
2,86229.152256,86505.723328,86772.234976,87033.009023,87227.137923,87291.7500
3,86252.304407,86528.949783,86795.532988,87056.377052,87250.558075,87315.1875
4,86309.351308,86586.179768,86852.939291,87113.955876,87308.265329,87372.9375
5,86336.763456,86613.679890,86880.524137,87141.623622,87335.994789,87400.6875
6,86377.511242,86654.558451,86921.528638,87182.751353,87377.214256,87441.9375
7,86297.559146,86574.349760,86841.072836,87102.053760,87296.336665,87361.0000
8,86364.669515,86641.675511,86908.606008,87169.789886,87364.223879,87428.9375
9,86437.336401,86714.575610,86981.730702,87243.134340,87437.731929,87502.5000


In [45]:
### 2) Connect model levels and surface values for temperature
air_temperature_0m = fn.variables['air_temperature_0m']
air_temperature_ml = fn.variables['air_temperature_ml']
### Temperature
temperature_ml = np.concatenate((air_temperature_ml[:,:,0,y[0],x[0]],air_temperature_0m[:,:,0,y[0],x[0]]),axis=1)

In [46]:
temperature_ml[:,:].shape

(67, 6)

In [47]:
pressure = p_ml
#temperature = air_temperature_ml[:,:,0,y[0],x[0]]
temperature = temperature_ml

In [48]:
pd.DataFrame(pressure)

,0,1,2,3,4,5
0,86153.336678,86429.664431,86695.941752,86956.486517,87150.444732,87215.0000
1,86240.697462,86517.305587,86783.852918,87044.661880,87238.816772,87303.4375
2,86229.152256,86505.723328,86772.234976,87033.009023,87227.137923,87291.7500
3,86252.304407,86528.949783,86795.532988,87056.377052,87250.558075,87315.1875
4,86309.351308,86586.179768,86852.939291,87113.955876,87308.265329,87372.9375
5,86336.763456,86613.679890,86880.524137,87141.623622,87335.994789,87400.6875
6,86377.511242,86654.558451,86921.528638,87182.751353,87377.214256,87441.9375
7,86297.559146,86574.349760,86841.072836,87102.053760,87296.336665,87361.0000
8,86364.669515,86641.675511,86908.606008,87169.789886,87364.223879,87428.9375
9,86437.336401,86714.575610,86981.730702,87243.134340,87437.731929,87502.5000


In [49]:
pd.DataFrame(temperature)

,0,1,2,3,4,5
0,272.740,272.936,273.117,273.270,273.325,271.345276
1,272.292,272.485,272.664,272.819,272.876,271.791595
2,271.971,272.176,272.367,272.535,272.649,271.817932
3,271.119,271.349,271.558,271.745,271.918,271.340637
4,270.313,270.540,270.746,270.925,271.068,270.634766
5,269.661,269.865,270.056,270.234,270.355,270.143677
6,269.341,269.561,269.770,269.959,270.121,270.138214
7,269.804,270.018,270.213,270.380,270.505,270.478699
8,269.190,269.420,269.635,269.840,270.016,270.105865
9,268.627,268.854,269.067,269.261,269.424,269.436340


In [ ]:
temperature.shape[1]-1

In [55]:
### 3) to convert pressure-levels into actual heights use the hypsometric equation --> Temperature and pressure
# are needed. After J. E. Martin: Mid-Latitude Atmospheric Dynamics Eq. 3.6
Rd = 287.    # gas constant for dry air [J kg^-1 K^-1]
g = 9.81     # Standard gravity [m s^-2]
    
### calculate the pressure-weighted, column averaged temperature as from J. E. Martin: Mid-Latitude Atmospheric 
# Dynamics Book, Eq. below Eq. 3.6
temp_mean = []
for i in range(0, temperature.shape[1]-1):
    numT = simps(y=temperature[:,i:(i+2)], x=np.log(pressure[:,i:(i+2)]), dx = np.log(pressure[:,i:(i+2)]),even='last')
    denomT = simps(y=np.ones(temperature[:,i:(i+2)].shape), x = np.log(pressure[:,i:(i+2)]), dx = np.log(pressure[:,i:(i+2)]),even='last')
    t_mean = numT/denomT
    temp_mean.append(t_mean)

In [56]:
# get temperature and pressure, and value so that array zero contains low levels (transpose or flip)
temp_mean = (np.fliplr(np.transpose(temp_mean)))
pressure = np.fliplr(pressure)
temperature = np.fliplr(temperature)

In [57]:
pressure.shape

(67, 6)

In [59]:
pd.DataFrame(pressure)

,0,1,2,3,4,5
0,87215.0000,87150.444732,86956.486517,86695.941752,86429.664431,86153.336678
1,87303.4375,87238.816772,87044.661880,86783.852918,86517.305587,86240.697462
2,87291.7500,87227.137923,87033.009023,86772.234976,86505.723328,86229.152256
3,87315.1875,87250.558075,87056.377052,86795.532988,86528.949783,86252.304407
4,87372.9375,87308.265329,87113.955876,86852.939291,86586.179768,86309.351308
5,87400.6875,87335.994789,87141.623622,86880.524137,86613.679890,86336.763456
6,87441.9375,87377.214256,87182.751353,86921.528638,86654.558451,86377.511242
7,87361.0000,87296.336665,87102.053760,86841.072836,86574.349760,86297.559146
8,87428.9375,87364.223879,87169.789886,86908.606008,86641.675511,86364.669515
9,87502.5000,87437.731929,87243.134340,86981.730702,86714.575610,86437.336401


In [58]:
temp_mean.shape

(67, 5)

In [60]:
pd.DataFrame(temp_mean)

,0,1,2,3,4
0,272.335138,273.2975,273.1935,273.0265,272.8380
1,272.333798,272.8475,272.7415,272.5745,272.3885
2,272.233466,272.5920,272.4510,272.2715,272.0735
3,271.629319,271.8315,271.6515,271.4535,271.2340
4,270.851383,270.9965,270.8355,270.6430,270.4265
5,270.249338,270.2945,270.1450,269.9605,269.7630
6,270.129607,270.0400,269.8645,269.6655,269.4510
7,270.491849,270.4425,270.2965,270.1155,269.9110
8,270.060933,269.9280,269.7375,269.5275,269.3050
9,269.430170,269.3425,269.1640,268.9605,268.7405


In [62]:
# calculate layer thickness
thickness = []
geop_th = []
for i in range(0, temp_mean.shape[1]):
   # if (i+1) == pres.shape[1]:
    #    continue
    p1 = pressure[:,i]
    p2 = pressure[:,(i+1)]   
    dz = (Rd * temp_mean[:,i])/g * np.log((p1/p2))    # thickness in [m]
    dgeop = (Rd * temp_mean[:,i])* np.log((p1/p2))    # thickness in [J/kg]
    thickness.append(dz)
    geop_th.append(dgeop)

In [67]:
pd.DataFrame(thickness)

,0,1,2,3,4
0,5.899536,17.814385,23.983638,24.570891,25.560809
1,5.899507,17.785053,23.943957,24.530213,25.518714
2,5.897333,17.768399,23.918454,24.502945,25.489201
3,5.884246,17.718827,23.848266,24.429329,25.410557
4,5.867394,17.664399,23.776630,24.356389,25.334916
5,5.854352,17.618640,23.716011,24.294968,25.272761
6,5.851758,17.602051,23.691386,24.268419,25.243539
7,5.859605,17.628287,23.729311,24.308917,25.286619
8,5.850270,17.594751,23.680236,24.256000,25.229858
9,5.836606,17.556586,23.629889,24.204973,25.176986


In [64]:
# transpose array, so that array starts at lower levels
thickness = np.transpose(thickness)
geop_th = np.transpose(geop_th)

In [70]:
pd.DataFrame(np.sum(thickness, axis=1))

,0
0,97.829260
1,97.677444
2,97.576332
3,97.291225
4,96.999727
5,96.756731
6,96.657153
7,96.812740
8,96.611115
9,96.405040
